# 🎯 Exercise Solution: Image Generation Agent with Cost Approval

This notebook implements an image generation agent that:
- Auto-approves single image requests (1 image)
- Pauses and asks for approval for bulk requests (>1 image)
- Uses the Everything MCP Server for image generation

## Setup

In [1]:
import os
from dotenv import load_dotenv
import uuid

# Load API key
load_dotenv()
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

print("✅ Environment setup complete")

✅ Environment setup complete


In [3]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully")

✅ ADK components imported successfully


In [4]:
# Configure retry options
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Retry configuration set")

✅ Retry configuration set


## Step 1: Create MCP Image Server Tool

In [5]:
# MCP integration with Everything Server for image generation
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Image Server created")

✅ MCP Image Server created


## Step 2: Create Long-Running Image Generation Tool

In [6]:
# Threshold for bulk image generation
BULK_IMAGE_THRESHOLD = 1

def generate_images(
    num_images: int,
    description: str,
    tool_context: ToolContext
) -> dict:
    """Generates images. Requires approval if requesting more than 1 image.
    
    Args:
        num_images: Number of images to generate
        description: Description of images to generate
        tool_context: ADK tool context for approval handling
        
    Returns:
        Dictionary with generation status
    """
    
    # SCENARIO 1: Single image (≤1) - auto-approve
    if num_images <= BULK_IMAGE_THRESHOLD:
        return {
            "status": "approved",
            "num_images": num_images,
            "description": description,
            "message": f"Auto-approved: Generating {num_images} image(s)",
            "should_generate": True
        }
    
    # SCENARIO 2: Bulk request - FIRST CALL - Request approval
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Bulk image generation: {num_images} images. This may incur additional costs. Approve?",
            payload={"num_images": num_images, "description": description}
        )
        return {
            "status": "pending",
            "message": f"Request for {num_images} images requires approval",
            "should_generate": False
        }
    
    # SCENARIO 3: RESUMED CALL - Handle approval decision
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "num_images": num_images,
            "description": description,
            "message": f"Approved: Generating {num_images} images",
            "should_generate": True
        }
    else:
        return {
            "status": "rejected",
            "message": f"Bulk image generation rejected: {num_images} images",
            "should_generate": False
        }

print("✅ Long-running image generation tool created")

✅ Long-running image generation tool created


## Step 3: Create Image Generation Agent with Both Tools

In [7]:
# Create image generation agent with approval logic and MCP tool
image_gen_agent = LlmAgent(
    name="image_generation_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are an image generation assistant with cost controls.
    
    When users request images:
    1. First, use the generate_images tool to check approval status
    2. If status is 'pending', inform the user that approval is required
    3. If status is 'approved' and should_generate is True, use the MCP getTinyImage tool to generate the requested number of images
    4. If status is 'rejected', inform the user the request was declined
    5. Provide clear feedback about what was generated or why it wasn't
    
    Note: The getTinyImage tool generates one image per call. Call it multiple times for bulk requests.
    """,
    tools=[
        FunctionTool(func=generate_images),
        mcp_image_server
    ],
)

print("✅ Image generation agent created")

✅ Image generation agent created


## Step 4: Wrap in Resumable App and Create Runner

In [8]:
# Wrap agent in resumable app
image_gen_app = App(
    name="image_generator",
    root_agent=image_gen_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

# Create session service and runner
session_service = InMemorySessionService()

image_gen_runner = Runner(
    app=image_gen_app,
    session_service=session_service,
)

print("✅ Resumable app and runner created")

✅ Resumable app and runner created


/var/folders/9w/xx7fv3jn75v2g1gjg6bjwrm40000gn/T/ipykernel_61722/1796130856.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


## Step 5: Helper Functions for Workflow

In [9]:
def check_for_approval(events):
    """Check if events contain an approval request."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )

print("✅ Helper functions defined")

✅ Helper functions defined


## Step 6: Main Workflow Function

In [10]:
async def run_image_generation_workflow(query: str, auto_approve: bool = True):
    """Runs an image generation workflow with approval handling.
    
    Args:
        query: User's image generation request
        auto_approve: Whether to auto-approve bulk requests (simulates human decision)
    """
    
    print(f"\n{'='*70}")
    print(f"User > {query}\n")
    
    # Generate unique session ID
    session_id = f"image_gen_{uuid.uuid4().hex[:8]}"
    
    # Create session
    await session_service.create_session(
        app_name="image_generator",
        user_id="test_user",
        session_id=session_id
    )
    
    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []
    
    # STEP 1: Send initial request to the agent
    async for event in image_gen_runner.run_async(
        user_id="test_user",
        session_id=session_id,
        new_message=query_content
    ):
        events.append(event)
    
    # STEP 2: Check if approval is needed
    approval_info = check_for_approval(events)
    
    # STEP 3: Handle approval workflow if needed
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")
        
        # Resume with approval decision
        async for event in image_gen_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(approval_info, auto_approve),
            invocation_id=approval_info["invocation_id"]
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")
    else:
        # No approval needed - display response
        print_agent_response(events)
    
    print(f"{'='*70}\n")

print("✅ Workflow function ready")

✅ Workflow function ready


## Step 7: Test the Image Generation Agent

In [11]:
# Demo 1: Single image request - auto-approved
await run_image_generation_workflow("Generate 1 sample image")


User > Generate 1 sample image



/Users/xiaowei.wan/.pyenv/versions/kuma-agent-env/lib/python3.13/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


Agent > Here is a sample image. 




In [12]:
# Demo 2: Bulk request with approval
await run_image_generation_workflow(
    "Generate 3 sample images",
    auto_approve=True
)


User > Generate 3 sample images



/Users/xiaowei.wan/.pyenv/versions/kuma-agent-env/lib/python3.13/site-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/Users/xiaowei.wan/.pyenv/versions/kuma-agent-env/lib/python3.13/site-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅



Agent > I have generated 3 sample images for you.



In [13]:
# Demo 3: Bulk request rejected
await run_image_generation_workflow(
    "Generate 5 sample images",
    auto_approve=False
)


User > Generate 5 sample images



⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > I'm sorry, but your request for 5 images has been rejected. Please request 1 image at a time.



## Summary

This exercise demonstrates:

1. **MCP Integration**: Used the Everything MCP Server for image generation
2. **Long-Running Operations**: Implemented approval logic that pauses for bulk requests
3. **Resumable Workflows**: Created a workflow that maintains state across approval requests
4. **Multi-Tool Agents**: Combined a custom function tool with an MCP tool

### Key Patterns Used:

- ✅ `ToolContext` and `request_confirmation()` for approval requests
- ✅ `App` with `ResumabilityConfig` for stateful workflows
- ✅ `invocation_id` to resume paused executions
- ✅ `McpToolset` for external service integration

### Production Considerations:

For a real production system, you would:
- Replace the Everything MCP Server with a real image generation service (DALL-E, Stable Diffusion, etc.)
- Add cost calculation based on actual API pricing
- Implement a real approval UI instead of simulated decisions
- Add image display functionality to show generated results
- Include error handling for failed generations
- Store generated images with proper metadata